<a href="https://colab.research.google.com/github/GabrielArcanjo97/classificacao_com_spark/blob/main/Modelo_de_classificacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=9a8959d9b267f91153cc4b2e0016e8aec11def2067476c0daf0dad1bf1e17036
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.sql import session

spark = session.SparkSession.builder\
        .master('local[*]')\
        .appName('Classificação')\
        .getOrCreate()

spark

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


**Importando os Dados**

In [18]:
dados = spark.read.csv('/content/drive/MyDrive/Alura/BIG DATA/Spark: criando modelos de classificação/base de dados/base de dados', header=True, inferSchema=True)

In [19]:
dados.show()

+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+----------------+-------------+
| id|Churn|Mais65anos|Conjuge|Dependentes|MesesDeContrato|TelefoneFixo|MaisDeUmaLinhaTelefonica|   Internet|   SegurancaOnline|      BackupOnline| SeguroDispositivo|    SuporteTecnico|           TVaCabo|   StreamingFilmes|TipoContrato|ContaCorreio| MetodoPagamento|MesesCobrados|
+---+-----+----------+-------+-----------+---------------+------------+------------------------+-----------+------------------+------------------+------------------+------------------+------------------+------------------+------------+------------+----------------+-------------+
|  0|  Nao|         0|    Sim|        Nao|              1|         Nao|    SemServicoTelefonico|        DSL|               Nao|               Sim|              

In [20]:
dados.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Churn: string (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- Conjuge: string (nullable = true)
 |-- Dependentes: string (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- TelefoneFixo: string (nullable = true)
 |-- MaisDeUmaLinhaTelefonica: string (nullable = true)
 |-- Internet: string (nullable = true)
 |-- SegurancaOnline: string (nullable = true)
 |-- BackupOnline: string (nullable = true)
 |-- SeguroDispositivo: string (nullable = true)
 |-- SuporteTecnico: string (nullable = true)
 |-- TVaCabo: string (nullable = true)
 |-- StreamingFilmes: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- ContaCorreio: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)



**Transformando colunas string em Binárias**

In [21]:
colunas_binarias = [
    'Churn',
    'Conjuge',
    'Dependentes',
    'TelefoneFixo',
    'MaisDeUmaLinhaTelefonica',
    'SegurancaOnline',
    'BackupOnline',
    'SeguroDispositivo',
    'SuporteTecnico',
    'TVaCabo',
    'StreamingFilmes',
    'ContaCorreio'
]

In [22]:
from pyspark.sql import functions as f

In [23]:
todas_as_colunas = [f.when(f.col(c)=='Sim',1).otherwise(0).alias(c) for c in colunas_binarias]

In [24]:
for colunas in reversed(dados.columns):
  if colunas not in colunas_binarias:
    todas_as_colunas.insert(0,colunas)

In [25]:
dataset = dados.select(todas_as_colunas)

In [26]:
dataset.show()

+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
| id|Mais65anos|MesesDeContrato|   Internet|TipoContrato| MetodoPagamento|MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|
+---+----------+---------------+-----------+------------+----------------+-------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+
|  0|         0|              1|        DSL| Mensalmente|BoletoEletronico|        29.85|    0|      1|          0|           0|                       0|              0|           1|                0|             0|      0|              0|      

**Utilizando dummies para transformação das colunas restantes**

In [27]:
Internet = dataset.groupby('id').pivot('Internet').agg(f.lit(1)).na.fill(0)
TipoContrato = dataset.groupby('id').pivot('TipoContrato').agg(f.lit(1)).na.fill(0)
MetodoPagamento = dataset.groupby('id').pivot('MetodoPagamento').agg(f.lit(1)).na.fill(0)

**Juntando colunas criadas com o dataset**

In [28]:
MetodoPagamento.show(3)

+----+------+----------------+-------------+-------------+
|  id|Boleto|BoletoEletronico|CartaoCredito|DebitoEmConta|
+----+------+----------------+-------------+-------------+
|3997|     0|               0|            1|            0|
|7554|     0|               1|            0|            0|
|6336|     0|               1|            0|            0|
+----+------+----------------+-------------+-------------+
only showing top 3 rows



In [29]:
dataset = dataset\
    .join(Internet, 'id', how='inner')\
    .join(TipoContrato, 'id', how='inner')\
    .join(MetodoPagamento, 'id', how='inner')\
    .select(
        '*',
        f.col('DSL').alias('Internet_DSL'),
        f.col('FibraOptica').alias('Internet_FibraOptica'),
        f.col('Nao').alias('Internet_Nao'),
        f.col('DoisAnos').alias('TipoContrato_DoisAnos'),
        f.col('Mensalmente').alias('TipoContrato_Mensalmente'),
        f.col('UmAno').alias('TipoContrato_UmAno'),
        f.col('Boleto').alias('MetodoPagamento_Boleto'),
        f.col('BoletoEletronico').alias('MetodoPagamento_BoletoEletronico'),
        f.col('CartaoCredito').alias('MetodoPagamento_CartaoCredito'),
        f.col('DebitoEmConta').alias('MetodoPagamento_DebitoEmConta')
    ).drop(
        'DSL','FibraOptica','Nao','DoisAnos','Mensalmente','UmAno','Boleto',
        'BoletoEletronico','CartaoCredito','DebitoEmConta','Internet','TipoContrato',
        'MetodoPagamento'
    )

In [30]:
dataset.show()

+----+----------+---------------+-----------------+-----+-------+-----------+------------+------------------------+---------------+------------+-----------------+--------------+-------+---------------+------------+------------+--------------------+------------+---------------------+------------------------+------------------+----------------------+--------------------------------+-----------------------------+-----------------------------+
|  id|Mais65anos|MesesDeContrato|    MesesCobrados|Churn|Conjuge|Dependentes|TelefoneFixo|MaisDeUmaLinhaTelefonica|SegurancaOnline|BackupOnline|SeguroDispositivo|SuporteTecnico|TVaCabo|StreamingFilmes|ContaCorreio|Internet_DSL|Internet_FibraOptica|Internet_Nao|TipoContrato_DoisAnos|TipoContrato_Mensalmente|TipoContrato_UmAno|MetodoPagamento_Boleto|MetodoPagamento_BoletoEletronico|MetodoPagamento_CartaoCredito|MetodoPagamento_DebitoEmConta|
+----+----------+---------------+-----------------+-----+-------+-----------+------------+----------------------

**Vetorizando os Dados**

In [31]:
from pyspark.ml.feature import VectorAssembler

In [33]:
dataset = dataset.withColumnRenamed('Churn','label')

In [34]:
X = dataset.columns
X.remove('id'),
X.remove('label')

In [35]:
X

['Mais65anos',
 'MesesDeContrato',
 'MesesCobrados',
 'Conjuge',
 'Dependentes',
 'TelefoneFixo',
 'MaisDeUmaLinhaTelefonica',
 'SegurancaOnline',
 'BackupOnline',
 'SeguroDispositivo',
 'SuporteTecnico',
 'TVaCabo',
 'StreamingFilmes',
 'ContaCorreio',
 'Internet_DSL',
 'Internet_FibraOptica',
 'Internet_Nao',
 'TipoContrato_DoisAnos',
 'TipoContrato_Mensalmente',
 'TipoContrato_UmAno',
 'MetodoPagamento_Boleto',
 'MetodoPagamento_BoletoEletronico',
 'MetodoPagamento_CartaoCredito',
 'MetodoPagamento_DebitoEmConta']

In [36]:
assembler = VectorAssembler(inputCols=X, outputCol='features')

In [37]:
dataset_prep = assembler.transform(dataset).select('features','label')

In [39]:
dataset_prep.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                   |label|
+-----------------------------------------------------------------------------------------------------------+-----+
|(24,[1,2,11,12,13,14,18,21],[1.0,45.30540797610398,1.0,1.0,1.0,1.0,1.0,1.0])                               |1    |
|(24,[1,2,3,5,6,8,9,11,12,13,15,18,21],[60.0,103.6142230120257,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|1    |
|(24,[1,2,5,6,10,11,12,13,14,19,20],[12.0,75.85,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                       |0    |
|(24,[1,2,3,5,8,12,13,14,17,22],[69.0,61.45,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                               |0    |
|(24,[1,2,3,5,6,11,13,15,18,21],[7.0,86.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                                 |1    |
|(24,[1,2,5,6,12,13,15,18,21],[14.0,85.03742670311915,1.0,1.0,1.0,1.0,1.

#**Modelos de Classificação**

Estarei utilizando três tipos de modelo de classificação, LogisticRegression, DecisionTreeClassifier e RandomForestClassifier

**Separando dataset entre treino e teste**

In [41]:
treino, teste = dataset_prep.randomSplit([0.7, 0.3], seed=101)

##**Regressão Logística**

In [40]:
from pyspark.ml.classification import LogisticRegression

In [43]:
lr = LogisticRegression()

In [44]:
modelo_lr = lr.fit(treino)

In [45]:
resumo_lr_treino = modelo_lr.transform(treino)

In [46]:
resumo_lr_treino.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[2.73984345620471...|[0.93933717698701...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[3.12234923512770...|[0.95780527310256...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[0.67500316482200...|[0.66262254467877...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[1.00825714289874...|[0.73267893080852...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[0.92173527128093...|[0.71539554720767...|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[-0.8138780856902...|[0.30706471540060...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[0.76468935125143...|[0.68237097574113...|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[0.08473036380618...|[0.52116992714404...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[0.58942102297452...|[0.64323229045600...|       0.0|
|(24,[0,1,2,3,4,

In [56]:
resumo_lr_teste = modelo_lr.transform(teste)

**Ajustando Métricas de Treino**

In [51]:
ajuste_lr_treino = modelo_lr.summary

In [52]:
print('Acurácia: %f'% ajuste_lr_treino.accuracy)
print('Precisão: %f'% ajuste_lr_treino.precisionByLabel[1])
print('Recall: %f'% ajuste_lr_treino.recallByLabel[1])
print('F1 Score: %f'% ajuste_lr_treino.fMeasureByLabel()[1])

Acurácia: 0.783791
Precisão: 0.768162
Recall: 0.812934
F1 Score: 0.789914


**Ajustando Métricas de Teste**

In [53]:
ajuste_lr_teste = modelo_lr.evaluate(teste)

In [54]:
print('Acurácia: %f'% ajuste_lr_teste.accuracy)
print('Precisão: %f'% ajuste_lr_teste.precisionByLabel[1])
print('Recall: %f'% ajuste_lr_teste.recallByLabel[1])
print('F1 Score: %f'% ajuste_lr_teste.fMeasureByLabel()[1])

Acurácia: 0.774984
Precisão: 0.760870
Recall: 0.802037
F1 Score: 0.780911


**Calculando Matriz de Confusão Treino e Teste**

In [55]:
tp = resumo_lr_treino.select('label','prediction').where((f.col('label')==1)&(f.col('prediction')==1)).count()
tn = resumo_lr_treino.select('label','prediction').where((f.col('label')==0)&(f.col('prediction')==0)).count()
fp = resumo_lr_treino.select('label','prediction').where((f.col('label')==1)&(f.col('prediction')==0)).count()
fn = resumo_lr_treino.select('label','prediction').where((f.col('label')==0)&(f.col('prediction')==1)).count()

print(tp, tn, fp, fn)

2929 2719 674 884


In [57]:
tp1 = resumo_lr_teste.select('label','prediction').where((f.col('label')==1)&(f.col('prediction')==1)).count()
tn1 = resumo_lr_teste.select('label','prediction').where((f.col('label')==0)&(f.col('prediction')==0)).count()
fp1 = resumo_lr_teste.select('label','prediction').where((f.col('label')==1)&(f.col('prediction')==0)).count()
fn1 = resumo_lr_teste.select('label','prediction').where((f.col('label')==0)&(f.col('prediction')==1)).count()

print(tp1, tn1, fp1, fn1)

1260 1175 311 396


**Função para melhor visualização das métricas e matriz de confusão**

In [58]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

def comprar_metricas(lista_previsoes):
  s = '\n'

  for modelo, df_transform_modelo in lista_previsoes.items():
    s += '-' * 50 + '\n'
    s += modelo + '\n'

    # montando matriz de confusão

    tp = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 1)).count()
    tn = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 0) & (f.col('prediction') == 0)).count()
    fp = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 0) & (f.col('prediction') == 1)).count()
    fn = df_transform_modelo.select('label', 'prediction').where((f.col('label') == 1) & (f.col('prediction') == 0)).count()

    # adiciono os valores de cada métrica a minha string de retorno com MulticlassClassificationEvaluator
    evaluator = MulticlassClassificationEvaluator()

    s += f'Acurácia: {evaluator.evaluate(df_transform_modelo, {evaluator.metricName: "accuracy"})}\n'
    s += f'Precisão: {evaluator.evaluate(df_transform_modelo, {evaluator.metricName: "precisionByLabel", evaluator.metricLabel: 1})}\n'
    s += f'Recall: {evaluator.evaluate(df_transform_modelo, {evaluator.metricName: "recallByLabel", evaluator.metricLabel: 1})}\n'
    s += f'F1: {evaluator.evaluate(df_transform_modelo, {evaluator.metricName: "fMeasureByLabel", evaluator.metricLabel: 1})}\n'
    s += ' '*20 + 'Previsto\n'
    s += ' '*15 +  'Churn' + ' '*5 + 'Não-Churn\n'
    s += ' '*4 + 'Churn' + ' '*6 +  str(int(tp)) + ' '*7 + str(int(fn)) + '\n'
    s += 'Real\n'
    s += ' '*4 + 'Não-Churn' + ' '*2 + str(int(fp)) +  ' '*7 + str(int(tn))  + '\n'
    s += '\n'

  return s

## **Árvore de Classificação**

In [59]:
from pyspark.ml.classification import DecisionTreeClassifier

In [60]:
dtc = DecisionTreeClassifier(seed=101, maxDepth=7)

In [62]:
modelo_dtc = dtc.fit(treino)

In [63]:
ajuste_dtc_treino = modelo_dtc.transform(treino)

In [64]:
ajuste_dtc_treino.show()

+--------------------+-----+--------------+--------------------+----------+
|            features|label| rawPrediction|         probability|prediction|
+--------------------+-----+--------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|    [77.0,6.0]|[0.92771084337349...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|     [5.0,0.0]|           [1.0,0.0]|       0.0|
|(24,[0,1,2,3,4,5,...|    0|    [12.0,0.0]|           [1.0,0.0]|       0.0|
|(24,[0,1,2,3,4,5,...|    0|    [12.0,0.0]|           [1.0,0.0]|       0.0|
|(24,[0,1,2,3,4,5,...|    0|    [12.0,0.0]|           [1.0,0.0]|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[234.0,1598.0]|[0.12772925764192...|       1.0|
|(24,[0,1,2,3,4,5,...|    0| [148.0,171.0]|[0.46394984326018...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[234.0,1598.0]|[0.12772925764192...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[234.0,1598.0]|[0.12772925764192...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[234.0,1598.0]|[0.12772925764192...|       1.0|
|(24,[0,1,2,

In [65]:
ajuste_dtc_teste = modelo_dtc.transform(teste)

**Ajustando Métricas Manualmente**

In [72]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [73]:
evaluator = MulticlassClassificationEvaluator()

In [75]:
# Treino

print('Acuracia: %f'% evaluator.evaluate(ajuste_dtc_treino, {evaluator.metricName:'accuracy'}))
print('Precisão: %f'% evaluator.evaluate(ajuste_dtc_treino, {evaluator.metricName:'precisionByLabel', evaluator.metricLabel:1}))
print('Recall: %f'% evaluator.evaluate(ajuste_dtc_treino, {evaluator.metricName:'recallByLabel', evaluator.metricLabel:1}))
print('F1: %f'% evaluator.evaluate(ajuste_dtc_treino, {evaluator.metricName:'fMeasureByLabel', evaluator.metricLabel:1}))

Acuracia: 0.815432
Precisão: 0.789702
Recall: 0.859839
F1: 0.823279


In [76]:
# Teste

print('Acuracia: %f'% evaluator.evaluate(ajuste_dtc_teste, {evaluator.metricName:'accuracy'}))
print('Precisão: %f'% evaluator.evaluate(ajuste_dtc_teste, {evaluator.metricName:'precisionByLabel', evaluator.metricLabel:1}))
print('Recall: %f'% evaluator.evaluate(ajuste_dtc_teste, {evaluator.metricName:'recallByLabel', evaluator.metricLabel:1}))
print('F1: %f'% evaluator.evaluate(ajuste_dtc_teste, {evaluator.metricName:'fMeasureByLabel', evaluator.metricLabel:1}))

Acuracia: 0.781668
Precisão: 0.761370
Recall: 0.820496
F1: 0.789828


In [77]:
# matriz de confusão treino

tp1 = ajuste_dtc_treino.select('label','prediction').where((f.col('label')==1)&(f.col('prediction')==1)).count()
tn1 = ajuste_dtc_treino.select('label','prediction').where((f.col('label')==0)&(f.col('prediction')==0)).count()
fp1 = ajuste_dtc_treino.select('label','prediction').where((f.col('label')==1)&(f.col('prediction')==0)).count()
fn1 = ajuste_dtc_treino.select('label','prediction').where((f.col('label')==0)&(f.col('prediction')==1)).count()

print(tp1, tn1, fp1, fn1)

3098 2778 505 825


In [78]:
# matriz de confusão teste

tp1 = ajuste_dtc_teste.select('label','prediction').where((f.col('label')==1)&(f.col('prediction')==1)).count()
tn1 = ajuste_dtc_teste.select('label','prediction').where((f.col('label')==0)&(f.col('prediction')==0)).count()
fp1 = ajuste_dtc_teste.select('label','prediction').where((f.col('label')==1)&(f.col('prediction')==0)).count()
fn1 = ajuste_dtc_teste.select('label','prediction').where((f.col('label')==0)&(f.col('prediction')==1)).count()

print(tp1, tn1, fp1, fn1)

1289 1167 282 404


**Ajustando Métricas com a Função Criada**

In [66]:
print(comprar_metricas({'Árvore de Decisão Treino':ajuste_dtc_treino,
                        'Árvore de Decisão Teste':ajuste_dtc_teste}))


--------------------------------------------------
Árvore de Decisão Treino
Acurácia: 0.8154315847904524
Precisão: 0.7897017588580169
Recall: 0.8598390230363586
F1: 0.823279298432102
                    Previsto
               Churn     Não-Churn
    Churn      3098       505
Real
    Não-Churn  825       2778

--------------------------------------------------
Árvore de Decisão Teste
Acurácia: 0.7816677275620624
Precisão: 0.761370348493798
Recall: 0.8204964990451942
F1: 0.7898284313725491
                    Previsto
               Churn     Não-Churn
    Churn      1289       282
Real
    Não-Churn  404       1167




Explicando a previsão:

O modelo de Árvore de decisão teste, preveu que 1289 foram previsto como Churn e realmente eram Churn, ao lado foram previstos que 282 não-Churn mas eram Churn.

Abaixo temos um total de 404 que foram previsto como não-Churn mas na verdade eram Churn, e 1167 como não-Churn e realmente não eram.

Em resumo, nosso modelo errou em 282 e 404.

## **Random Forest**

In [81]:
from pyspark.ml.classification import RandomForestClassifier

In [82]:
rfc = RandomForestClassifier(seed=101, maxDepth=7, numTrees=10)

In [83]:
modelo_rfc = rfc.fit(treino)

In [86]:
ajuste_rfc_treino = modelo_rfc.transform(treino)

ajuste_rfc_treino.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[8.28318068651483...|[0.82831806865148...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[8.60220234699535...|[0.86022023469953...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[6.16541610558594...|[0.61654161055859...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[6.87509352494078...|[0.68750935249407...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[6.38855502300594...|[0.63885550230059...|       0.0|
|(24,[0,1,2,3,4,5,...|    1|[2.88780576205113...|[0.28878057620511...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[3.66768768449392...|[0.36676876844939...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[2.37647312200955...|[0.23764731220095...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[2.37647312200955...|[0.23764731220095...|       1.0|
|(24,[0,1,2,3,4,

In [87]:
ajuste_rfc_teste = modelo_rfc.transform(teste)

ajuste_rfc_teste.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|[8.26331345810646...|[0.82633134581064...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|[4.23030553230402...|[0.42303055323040...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[3.35049409101441...|[0.33504940910144...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[1.85018095045766...|[0.18501809504576...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[3.19442270215773...|[0.31944227021577...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[2.39790244747695...|[0.23979024474769...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|[3.37840862897067...|[0.33784086289706...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[4.63053906310899...|[0.46305390631089...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|[5.74012620093250...|[0.57401262009325...|       0.0|
|(24,[0,1,2,3,4,

**Ajustanto Métricas Manualmente**

In [89]:
# Treino

print('Acuracia: %f'% evaluator.evaluate(ajuste_rfc_treino, {evaluator.metricName:'accuracy'}))
print('Precisão: %f'% evaluator.evaluate(ajuste_rfc_treino, {evaluator.metricName:'precisionByLabel', evaluator.metricLabel:1}))
print('Recall: %f'% evaluator.evaluate(ajuste_rfc_treino, {evaluator.metricName:'recallByLabel', evaluator.metricLabel:1}))
print('F1: %f'% evaluator.evaluate(ajuste_rfc_treino, {evaluator.metricName:'fMeasureByLabel', evaluator.metricLabel:1}))

Acuracia: 0.811130
Precisão: 0.789814
Recall: 0.847905
F1: 0.817829


In [90]:
# Teste

print('Acuracia: %f'% evaluator.evaluate(ajuste_rfc_teste, {evaluator.metricName:'accuracy'}))
print('Precisão: %f'% evaluator.evaluate(ajuste_rfc_teste, {evaluator.metricName:'precisionByLabel', evaluator.metricLabel:1}))
print('Recall: %f'% evaluator.evaluate(ajuste_rfc_teste, {evaluator.metricName:'recallByLabel', evaluator.metricLabel:1}))
print('F1: %f'% evaluator.evaluate(ajuste_rfc_teste, {evaluator.metricName:'fMeasureByLabel', evaluator.metricLabel:1}))

Acuracia: 0.796626
Precisão: 0.775414
Recall: 0.835137
F1: 0.804168


In [91]:
# matriz de confusão treino

tp3 = ajuste_rfc_treino.select('label','prediction').where((f.col('label')==1)&(f.col('prediction')==1)).count()
tn3 = ajuste_rfc_treino.select('label','prediction').where((f.col('label')==0)&(f.col('prediction')==0)).count()
fp3 = ajuste_rfc_treino.select('label','prediction').where((f.col('label')==1)&(f.col('prediction')==0)).count()
fn3 = ajuste_rfc_treino.select('label','prediction').where((f.col('label')==0)&(f.col('prediction')==1)).count()

print(tp3, tn3, fp3, fn3)

3055 2790 548 813


In [92]:
# matriz de confusão teste

tp4 = ajuste_rfc_teste.select('label','prediction').where((f.col('label')==1)&(f.col('prediction')==1)).count()
tn4 = ajuste_rfc_teste.select('label','prediction').where((f.col('label')==0)&(f.col('prediction')==0)).count()
fp4 = ajuste_rfc_teste.select('label','prediction').where((f.col('label')==1)&(f.col('prediction')==0)).count()
fn4 = ajuste_rfc_teste.select('label','prediction').where((f.col('label')==0)&(f.col('prediction')==1)).count()

print(tp4, tn4, fp4, fn4)

1312 1191 259 380


**Ajustando Métricas com a Função Criada**

In [93]:
print(comprar_metricas({'Random Forest Treino':ajuste_rfc_treino,
                        'Random Forest Teste':ajuste_rfc_teste}))


--------------------------------------------------
Random Forest Treino
Acurácia: 0.8111296142103802
Precisão: 0.7898138572905895
Recall: 0.8479045240077713
F1: 0.8178289385624413
                    Previsto
               Churn     Não-Churn
    Churn      3055       548
Real
    Não-Churn  813       2790

--------------------------------------------------
Random Forest Teste
Acurácia: 0.7966263526416295
Precisão: 0.7754137115839244
Recall: 0.8351368555060471
F1: 0.8041679436101747
                    Previsto
               Churn     Não-Churn
    Churn      1312       259
Real
    Não-Churn  380       1191




Explicando a previsão:

O modelo de Árvore de decisão teste, preveu que 1312 foram previsto como Churn e realmente eram Churn, ao lado foram previstos que 259 não-Churn mas eram Churn.

Abaixo temos um total de 380 que foram previsto como não-Churn mas na verdade eram Churn, e 1191 como não-Churn e realmente não eram.

Em resumo, nosso modelo errou em 259 e 380.

##**Comparando Árvore de Decisão e Random Forest**

In [94]:
print(comprar_metricas({'Árvore de Decisão Teste':ajuste_dtc_teste,
                        'Random Forest Teste':ajuste_rfc_teste}))


--------------------------------------------------
Árvore de Decisão Teste
Acurácia: 0.7816677275620624
Precisão: 0.761370348493798
Recall: 0.8204964990451942
F1: 0.7898284313725491
                    Previsto
               Churn     Não-Churn
    Churn      1289       282
Real
    Não-Churn  404       1167

--------------------------------------------------
Random Forest Teste
Acurácia: 0.7966263526416295
Precisão: 0.7754137115839244
Recall: 0.8351368555060471
F1: 0.8041679436101747
                    Previsto
               Churn     Não-Churn
    Churn      1312       259
Real
    Não-Churn  380       1191




Ao analisar o os dois modelos, percebe-se que o random forest está melhor do que o modelo de árvore de decisão.

##**Otimizando Modelos**

###**Árvore de Decisão**

In [95]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [96]:
dtc = DecisionTreeClassifier()

In [97]:
grid = ParamGridBuilder()\
      .addGrid(dtc.maxDepth,[2,5,10])\
      .addGrid(dtc.maxBins,[10,32,45])\
      .build()

In [98]:
evaluator = MulticlassClassificationEvaluator()

In [99]:
dtc_cv = CrossValidator(
    estimator=dtc,
    estimatorParamMaps=grid,
    evaluator = evaluator,
    numFolds=3,
    seed=101
)

**Ajustando Métricas**

In [100]:
modelo_dtc_cv = dtc_cv.fit(treino)

In [103]:
ajuste_dtc_cv = modelo_dtc_cv.transform(teste)

ajuste_dtc_cv.show()

+--------------------+-----+-------------+--------------------+----------+
|            features|label|rawPrediction|         probability|prediction|
+--------------------+-----+-------------+--------------------+----------+
|(24,[0,1,2,3,4,5,...|    0|   [13.0,1.0]|[0.92857142857142...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|  [45.0,76.0]|[0.37190082644628...|       1.0|
|(24,[0,1,2,3,4,5,...|    1|  [49.0,83.0]|[0.37121212121212...|       1.0|
|(24,[0,1,2,3,4,5,...|    0| [35.0,328.0]|[0.09641873278236...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|  [50.0,39.0]|[0.56179775280898...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|    [4.0,0.0]|           [1.0,0.0]|       0.0|
|(24,[0,1,2,3,4,5,...|    1|  [49.0,83.0]|[0.37121212121212...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|  [49.0,83.0]|[0.37121212121212...|       1.0|
|(24,[0,1,2,3,4,5,...|    0|   [24.0,2.0]|[0.92307692307692...|       0.0|
|(24,[0,1,2,3,4,5,...|    0|   [56.0,0.0]|           [1.0,0.0]|       0.0|
|(24,[0,1,2,3,4,5,...|   

In [104]:
print('Acurácia: %f'% evaluator.evaluate(ajuste_dtc_cv, {evaluator.metricName:'accuracy'}))
print('Precisão: %f'% evaluator.evaluate(ajuste_dtc_cv, {evaluator.metricName:'precisionByLabel', evaluator.metricLabel:1}))
print('Recall: %f'% evaluator.evaluate(ajuste_dtc_cv, {evaluator.metricName:'recallByLabel', evaluator.metricLabel:1}))
print('F1: %f'% evaluator.evaluate(ajuste_dtc_cv, {evaluator.metricName:'fMeasureByLabel', evaluator.metricLabel:1}))

Acurácia: 0.787715
Precisão: 0.772618
Recall: 0.815404
F1: 0.793434


In [105]:
tp = ajuste_dtc_cv.select('label','prediction').where((f.col('label')==1)&(f.col('prediction')==1)).count()
tn = ajuste_dtc_cv.select('label','prediction').where((f.col('label')==0)&(f.col('prediction')==0)).count()
fp = ajuste_dtc_cv.select('label','prediction').where((f.col('label')==1)&(f.col('prediction')==0)).count()
fn = ajuste_dtc_cv.select('label','prediction').where((f.col('label')==0)&(f.col('prediction')==1)).count()

print(tp, tn, fp, fn)

1281 1194 290 377


**Comparando métricas com a função criada**

Comparando metrica da Árvore de Decisão sem otimizar e com otimização

In [107]:
print(comprar_metricas({'Árvore de Decisão Sem Otimização':ajuste_dtc_teste,
                        'Árvore de Decisão Com Otimização':ajuste_dtc_cv}))


--------------------------------------------------
Árvore de Decisão Sem Otimização
Acurácia: 0.7816677275620624
Precisão: 0.761370348493798
Recall: 0.8204964990451942
F1: 0.7898284313725491
                    Previsto
               Churn     Não-Churn
    Churn      1289       282
Real
    Não-Churn  404       1167

--------------------------------------------------
Árvore de Decisão Com Otimização
Acurácia: 0.7877148313176321
Precisão: 0.7726176115802171
Recall: 0.8154042011457671
F1: 0.7934344998451532
                    Previsto
               Churn     Não-Churn
    Churn      1281       290
Real
    Não-Churn  377       1194




###**Random Forest**

In [108]:
from pyspark.ml.classification import RandomForestClassifier

In [109]:
rfc = RandomForestClassifier()

In [110]:
grid = ParamGridBuilder()\
      .addGrid(rfc.maxDepth,[2,5,10])\
      .addGrid(rfc.maxBins,[10,32,45])\
      .addGrid(rfc.numTrees,[10,20,50])\
      .build()

In [111]:
evaluator = MulticlassClassificationEvaluator()

In [112]:
rfc_cv = CrossValidator(
    estimator = rfc,
    estimatorParamMaps=grid,
    evaluator = evaluator,
    numFolds=3,
    seed=101
)

**Ajustando Métricas**

In [113]:
modelo_rfc_cv = rfc_cv.fit(treino)

In [115]:
ajuste_rfc_cv = modelo_rfc_cv.transform(teste)

In [116]:
print('Acurácia: %f'% evaluator.evaluate(ajuste_rfc_cv, {evaluator.metricName:'accuracy'}))
print('Precisão: %f'% evaluator.evaluate(ajuste_rfc_cv, {evaluator.metricName:'precisionByLabel', evaluator.metricLabel:1}))
print('Recall: %f'% evaluator.evaluate(ajuste_rfc_cv, {evaluator.metricName:'recallByLabel', evaluator.metricLabel:1}))
print('F1: %f'% evaluator.evaluate(ajuste_rfc_cv, {evaluator.metricName:'fMeasureByLabel', evaluator.metricLabel:1}))

Acurácia: 0.809039
Precisão: 0.790197
Recall: 0.841502
F1: 0.815043


In [117]:
tp = ajuste_rfc_cv.select('label','prediction').where((f.col('label')==1)&(f.col('prediction')==1)).count()
tn = ajuste_rfc_cv.select('label','prediction').where((f.col('label')==0)&(f.col('prediction')==0)).count()
fp = ajuste_rfc_cv.select('label','prediction').where((f.col('label')==1)&(f.col('prediction')==0)).count()
fn = ajuste_rfc_cv.select('label','prediction').where((f.col('label')==0)&(f.col('prediction')==1)).count()

print(tp, tn, fp, fn)

1322 1220 249 351


In [118]:
print(comprar_metricas({'Random Forest Sem Otimização':ajuste_rfc_teste,
                        'Random Forest Com Otimização':ajuste_rfc_cv}))


--------------------------------------------------
Random Forest Sem Otimização
Acurácia: 0.7966263526416295
Precisão: 0.7754137115839244
Recall: 0.8351368555060471
F1: 0.8041679436101747
                    Previsto
               Churn     Não-Churn
    Churn      1312       259
Real
    Não-Churn  380       1191

--------------------------------------------------
Random Forest Com Otimização
Acurácia: 0.8090388287714831
Precisão: 0.7901972504482965
Recall: 0.841502227880331
F1: 0.815043156596794
                    Previsto
               Churn     Não-Churn
    Churn      1322       249
Real
    Não-Churn  351       1220




##**Escolhendo o melhor modelo otimizado**

In [119]:
print(comprar_metricas({'Árvore de Decisão Com Otimização':ajuste_dtc_cv,
                        'Random Forest Com Otimização':ajuste_rfc_cv}))


--------------------------------------------------
Árvore de Decisão Com Otimização
Acurácia: 0.7877148313176321
Precisão: 0.7726176115802171
Recall: 0.8154042011457671
F1: 0.7934344998451532
                    Previsto
               Churn     Não-Churn
    Churn      1281       290
Real
    Não-Churn  377       1194

--------------------------------------------------
Random Forest Com Otimização
Acurácia: 0.8090388287714831
Precisão: 0.7901972504482965
Recall: 0.841502227880331
F1: 0.815043156596794
                    Previsto
               Churn     Não-Churn
    Churn      1322       249
Real
    Não-Churn  351       1220




O melhor modelo escolhido é o Random Forest, pois os resultados mostrados estão melhores do que a Arvore de Decisão.

Um cientista de dados não serve somente para rodar modelos, é necessário entender o que realmente o departamento quer, nesse caso ele quer saber se o cliete vai cancelar o serviço ou não.

Então tudo vai depender para qual modelo, metrico a gente vai olhar.

A proposta é criar um modelo para prever se um cliente vai se tornar Churn ou não a partir de dados histórico, utilizando pyspark.


##**Treinando o Modelo Escolhido**

In [120]:
melhor_modelo_rfc_cv = modelo_rfc_cv.bestModel

In [124]:
# explorando quais são os melhores parametros do random forest

print(melhor_modelo_rfc_cv.getMaxDepth())
print(melhor_modelo_rfc_cv.getMaxBins())
print(melhor_modelo_rfc_cv.getNumTrees)

10
45
50


In [125]:
rfc_melhor = RandomForestClassifier(seed=101, maxDepth=10, maxBins=45, numTrees=50)

In [126]:
melhor_rfc = rfc_melhor.fit(dataset_prep)

In [127]:
X

['Mais65anos',
 'MesesDeContrato',
 'MesesCobrados',
 'Conjuge',
 'Dependentes',
 'TelefoneFixo',
 'MaisDeUmaLinhaTelefonica',
 'SegurancaOnline',
 'BackupOnline',
 'SeguroDispositivo',
 'SuporteTecnico',
 'TVaCabo',
 'StreamingFilmes',
 'ContaCorreio',
 'Internet_DSL',
 'Internet_FibraOptica',
 'Internet_Nao',
 'TipoContrato_DoisAnos',
 'TipoContrato_Mensalmente',
 'TipoContrato_UmAno',
 'MetodoPagamento_Boleto',
 'MetodoPagamento_BoletoEletronico',
 'MetodoPagamento_CartaoCredito',
 'MetodoPagamento_DebitoEmConta']

In [128]:
novo_cliente = [{
 'Mais65anos':0,
 'MesesDeContrato':1,
 'MesesCobrados':45.30450797610398,
 'Conjuge':0,
 'Dependentes':0,
 'TelefoneFixo':0,
 'MaisDeUmaLinhaTelefonica':0,
 'SegurancaOnline':0,
 'BackupOnline':0,
 'SeguroDispositivo':0,
 'SuporteTecnico':0,
 'TVaCabo':1,
 'StreamingFilmes':1,
 'ContaCorreio':1,
 'Internet_DSL':1,
 'Internet_FibraOptica':1,
 'Internet_Nao':0,
 'TipoContrato_DoisAnos':0,
 'TipoContrato_Mensalmente':1,
 'TipoContrato_UmAno':0,
 'MetodoPagamento_Boleto':0,
 'MetodoPagamento_BoletoEletronico':1,
 'MetodoPagamento_CartaoCredito':0,
 'MetodoPagamento_DebitoEmConta':0
}]

In [129]:
novo_cliente = spark.createDataFrame(novo_cliente)

novo_cliente.show()

+------------+-------+------------+-----------+------------+--------------------+------------+----------+------------------------+-----------------+---------------+----------------------+--------------------------------+-----------------------------+-----------------------------+---------------+-----------------+---------------+--------------+-------+------------+---------------------+------------------------+------------------+
|BackupOnline|Conjuge|ContaCorreio|Dependentes|Internet_DSL|Internet_FibraOptica|Internet_Nao|Mais65anos|MaisDeUmaLinhaTelefonica|    MesesCobrados|MesesDeContrato|MetodoPagamento_Boleto|MetodoPagamento_BoletoEletronico|MetodoPagamento_CartaoCredito|MetodoPagamento_DebitoEmConta|SegurancaOnline|SeguroDispositivo|StreamingFilmes|SuporteTecnico|TVaCabo|TelefoneFixo|TipoContrato_DoisAnos|TipoContrato_Mensalmente|TipoContrato_UmAno|
+------------+-------+------------+-----------+------------+--------------------+------------+----------+------------------------+----

In [131]:
assembler = VectorAssembler(inputCols=X, outputCol='features')

In [134]:
novo_cliente_prep = assembler.transform(novo_cliente).select('features')

In [135]:
novo_cliente_prep.show(truncate=False)

+-----------------------------------------------------------------------------------+
|features                                                                           |
+-----------------------------------------------------------------------------------+
|(24,[1,2,11,12,13,14,15,18,21],[1.0,45.30450797610398,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
+-----------------------------------------------------------------------------------+



In [136]:
melhor_rfc.transform(novo_cliente_prep).show()

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|(24,[1,2,11,12,13...|[4.49925020579483...|[0.08998500411589...|       1.0|
+--------------------+--------------------+--------------------+----------+



De acordo com o melhor modelo escolhido, o qual foi Random Forest Otimizado, o novo cliente vai estar cancelando o plano.